# Quick background - the evasin family

Evasins are a family of secreted proteins found in ticks.

They bind to cytokines and modulate the host immune response.

The two well studied evasins, Evasin-1 and Evasin-4 from *Rhipicephalus sanguineus* are residues 114 and 127 residues long respectively.

The three-dimensional structure of Evasin-1 from *Rhipicephalus sanguineus* is known (PDB ID: `3FPR`). The family contains a conserved patterns of four disulpide bonds.

## Running external tools from within Python - a BLAST search

Let's run a BLAST search against the locally installed Uniprot database to find protein sequences similar to Evasin-1 and Evasin-4.

`evasins_canonical.fasta` contains the sequences for Evasin-1 and Evasin-4. We know from prior work that they are paralogous, so we will BLAST search them both and combine all the hits.


In [1]:
import subprocess
database = '/mnt/references/uniprot/2017_10/uniprot_2017_10'

# Since this takes ~5 min or more to run, we've prepared the "evasins_canonical_uniprot.blast7" output file earlier.
# Please don't run it, if everyone does you'll probably bring down the Jupyter server !

# process = subprocess.run('blastp -query evasins_canonical.fasta -out evasins_canonical_uniprot.blast7 -outfmt 7 -evalue 10 -db %s' % database, 
#                          shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# print(process)

In [2]:
# Instead, we will just download the results prepared earlier
from urllib.request import urlretrieve
github_base = "https://raw.githubusercontent.com/MonashBioinformaticsPlatform/intro-to-python-for-bioinformatics/master/notebooks"
urlretrieve("%s/evasins_canonical.fasta" % github_base, filename="evasins_canonical.fasta")
urlretrieve("%s/evasins_canonical_uniprot.blast7" % github_base, filename="evasins_canonical_uniprot.blast7")
# urllib.request.urlretrieve("%s/evasins_uniprot_hits.fasta.complete" % github_base, filename="evasins_uniprot_hits.fasta")

('evasins_canonical_uniprot.blast7', <http.client.HTTPMessage at 0x11193b0f0>)

BLAST+ format 7 (`-outfmt 7`) looks like this:

In [3]:
!head evasins_canonical_uniprot.blast7

# BLASTP 2.2.31+
# Query: sp|P0C8E7|EVA1_RHISA Evasin-1 OS=Rhipicephalus sanguineus PE=1 SV=1
# Database: /mnt/references/uniprot/2017_10/uniprot_2017_10
# Fields: query id, subject id, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 123 hits found
sp|P0C8E7|EVA1_RHISA	sp|P0C8E7|EVA1_RHISA	100.00	114	0	0	1	114	1	114	9e-78	237
sp|P0C8E7|EVA1_RHISA	tr|C9W1N8|C9W1N8_RHISA	99.12	114	1	0	1	114	1	114	3e-77	235
sp|P0C8E7|EVA1_RHISA	tr|A0A131Z694|A0A131Z694_RHIAP	57.01	107	41	2	13	114	19	125	5e-34	125
sp|P0C8E7|EVA1_RHISA	tr|A0A131ZAV1|A0A131ZAV1_RHIAP	64.44	90	31	1	23	111	18	107	6e-34	125
sp|P0C8E7|EVA1_RHISA	tr|A0A131X8U1|A0A131X8U1_9ACAR	56.18	89	38	1	25	113	1	88	3e-30	115


The first few lines (`#`) contain metadata about the search that we can skip for now.
Each BLAST hit is a single line with tab-seperated fields. The second field is the sequence ID we want.
We will grab all the hits irrespective of e-value.

First we will parse the `evasins_canonical_uniprot.blast7` file to create a list of hits.

Once we have the IDs of each hit, we can use `blastdbcmd` to extract the full length sequences from the BLAST database.

In [4]:
blast_hits = []
with open('evasins_canonical_uniprot.blast7', 'r') as fh:
    for line in fh:
        if line[0] != '#':
            blast_hits.append(line.split('\t')[1])

blast_hits[0:5]

['sp|P0C8E7|EVA1_RHISA',
 'tr|C9W1N8|C9W1N8_RHISA',
 'tr|A0A131Z694|A0A131Z694_RHIAP',
 'tr|A0A131ZAV1|A0A131ZAV1_RHIAP',
 'tr|A0A131X8U1|A0A131X8U1_9ACAR']

## Challenge

Rather than extracting the ID of the hits, can you also instead parse the E-value field and create a list of E-values ?

## Solution

In [5]:
e_values = []
with open('evasins_canonical_uniprot.blast7', 'r') as fh:
    for line in fh:
        if line[0] != '#':
            # hit_id = line.split('\t')[1]
            e = line.split('\t')[10]
            e_values.append(e)

e_values[0:5]

['9e-78', '3e-77', '5e-34', '6e-34', '3e-30']

**Protip**: The scikit-bio package has a BLAST parser that captures all the fields into a Pandas dataframe. The BioPython package can also parse BLAST XML output. If I wasn't trying to demonstrate basic file parsing, I'd probably use one of those packages instead.

http://scikit-bio.org/docs/latest/generated/skbio.io.format.blast7.html#module-skbio.io.format.blast7

We want to ensure there are no duplicate IDs in our list (since the `blastp` command was supplied with two query sequences, Evasin-1 and Evasin-4, the `evasins_canonical_uniprot.blast7` output will contain the results of two BLAST searches).

If we turn the `list` into a `set`, Python will automatically discard duplicate IDs.

In [6]:
print("Total number of BLAST hits:", len(blast_hits))

Total number of BLAST hits: 149


In [7]:
blast_hits = set(blast_hits)

In [8]:
print("Number of hits with duplicates removed:", len(blast_hits))

Number of hits with duplicates removed: 132


Now we use `blastdbcmd` with out list of unique IDs to extract the full FASTA format sequence from the BLAST database.

In [9]:
# We redirect to a file with '>' using the Unix shell command, rather than 
# capturing stdout in process.stdout
cmd = 'blastdbcmd -db %s -entry "%s" >evasins_uniprot_hits.fasta' % (database, ','.join(blast_hits))
cmd

'blastdbcmd -db /mnt/references/uniprot/2017_10/uniprot_2017_10 -entry "tr|A0A131YQC0|A0A131YQC0_RHIAP,tr|A0A0C9R5W8|A0A0C9R5W8_AMBAM,tr|A0A131YS46|A0A131YS46_RHIAP,tr|A0A023G9Q7|A0A023G9Q7_9ACAR,tr|A0A131Z694|A0A131Z694_RHIAP,tr|L7MAB0|L7MAB0_9ACAR,tr|G3MSY5|G3MSY5_9ACAR,tr|A0A131Z586|A0A131Z586_RHIAP,tr|A0A0C9S4J0|A0A0C9S4J0_AMBAM,tr|L7M8Z8|L7M8Z8_9ACAR,tr|A0A023FQ58|A0A023FQ58_9ACAR,tr|A0A023GCC3|A0A023GCC3_9ACAR,tr|A0A023G9N9|A0A023G9N9_9ACAR,tr|A0A131YUQ8|A0A131YUQ8_RHIAP,tr|A0A224Y3D0|A0A224Y3D0_9ACAR,tr|A0A023FCU0|A0A023FCU0_9ACAR,tr|A0A131YNV9|A0A131YNV9_RHIAP,sp|P0C8E9|EVA4_RHISA,tr|A0A224Y3D8|A0A224Y3D8_9ACAR,tr|A0A023G4G4|A0A023G4G4_9ACAR,tr|G3MIM7|G3MIM7_9ACAR,tr|A0A023FF01|A0A023FF01_9ACAR,tr|A0A023FEC6|A0A023FEC6_9ACAR,tr|A0A1E5SLG8|A0A1E5SLG8_9BACT,tr|A0A131X968|A0A131X968_9ACAR,tr|A0A023G2G7|A0A023G2G7_9ACAR,tr|A0A023G2M2|A0A023G2M2_9ACAR,tr|A0A023G6B6|A0A023G6B6_9ACAR,tr|A0A131XJ87|A0A131XJ87_9ACAR,tr|A0A023FQ65|A0A023FQ65_9ACAR,tr|A0A223FZ22|A0A223FZ22_RHIMP,tr|A0A023

In [10]:
import subprocess

process = subprocess.run(cmd, shell=True, stderr=subprocess.PIPE)

# If you'd like to skip this step (eg if you don't have blastdbcmd or a local Uniprot BLAST database available),
# you can run:
#
# !cp evasins_uniprot_hits.fasta.complete evasins_uniprot_hits.fasta
#
# to create a copy of the expected output sequences.

In [11]:
# Take a peek at the output of our `blastdbcmd`, in the file evasin1_uniprot.fasta
!head evasins_uniprot_hits.fasta

## Using the Python ecosystem: FASTA parsing with skbio.io

http://scikit-bio.org/docs/latest/io.html

Since you know how to parse a file now, you could go ahead and write a FASTA format parser to read in our new set of putative evasin sequences. However, for most common file formats a parser already exists to make your life easier. We are going to use the one `scikit-bio` provides (`Bio.SeqIO` from Biopython would be another reasonable option).

*If you are curious, you'll find a quick-n-dirty example of a hand written FASTA parser in the `DIY_FASTA.ipynb` notebook.*

In [12]:
!pip install scikit-bio==0.5.1

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
import skbio.io

In [14]:
seqs = []
with open('evasins_uniprot_hits.fasta') as fh:
    for seq in skbio.io.read(fh, format='fasta'):
        seqs.append(seq)

/Users/perry/.virtualenvs/jupyter/lib/python3.6/site-packages/skbio/io/registry.py:548: FormatIdentificationWarning: <_io.TextIOWrapper name='evasins_uniprot_hits.fasta' mode='r' encoding='UTF-8'> does not look like a fasta file
  % (file, fmt), FormatIdentificationWarning)


In [15]:
len(seqs)

0

In [16]:
print(repr(seqs[0]))

IndexError: list index out of range

In [17]:
type(seqs[0])

IndexError: list index out of range

In [18]:
help(seqs[0])

IndexError: list index out of range

Or, look at the nicely formatted version online: http://scikit-bio.org/docs/latest/generated/skbio.sequence.Sequence.html#skbio.sequence.Sequence

In [19]:
seqs[0].metadata

IndexError: list index out of range

In [20]:
# A numpy array of each character in the sequence
seqs[0].values

IndexError: list index out of range

In [21]:
# Sequence implements a magic __str__ method that allows it to be converted to a string representation
str(seqs[0])

IndexError: list index out of range

It's not uncommon for FASTA sequences of protein to include 'X' residues when a residue type is undetermined.

Let's see if there are any 'X' residues in sequences of our set.

In [22]:
x_seqs = []
for seq in seqs:
    if seq.count('X') >= 1:
        x_seqs.append(seq)
        
print(', '.join([s.metadata['id'] for s in x_seqs]))

print("Total sequences:", len(seqs))
print("Sequences with X residues", len(x_seqs))


Total sequences: 0
Sequences with X residues 0


## Challenge

Write a function `enough_cys` that will return a list of sequences with eight or more cysteine residues. (Hint: A cyteine residue is represented by a 'C' character).

Once the function is written, we should be able to type:

```python
cys_seqs = enough_cys(seqs, 8)
```

and get a list of sequences with 8+ cysteine residues.

## Solution

In [23]:
def enough_cys(sequences, min_cys=8):
    """
    Given a list of sequences (scikit-bio Sequence or strings), return a list
    with `min_cys` or more cysteine residues.
    """
    cys = []
    for seq in sequences:
        # https://docs.python.org/3.6/library/stdtypes.html#str.count
        if seq.count('C') >= min_cys:
            # print('>{seq_id}'.format(seq_id=seq_id))
            # print(seq)
            cys.append(seq)
    
    return cys

cys_seqs = enough_cys(seqs, 8)

print("Total sequences:", len(seqs))
print("Sequences with >= 8 cyteines", len(cys_seqs))

# Bonus: You can use `assert` with some condition to test expected outputs of your function
# If any assertion fails, an error is raised. Otherwise they just execute silently.
assert len(enough_cys(["M" + "CA"*10, "MCACAC"])) == 1
assert len(enough_cys(["M" + "CA"*10, "MCACAC"], min_cys=3)) == 2
assert "MCACAC" not in enough_cys(["M" + "CA"*10, "MCACAC"])

Total sequences: 0
Sequences with >= 8 cyteines 0


We can also write a set of sequences back to a file

In [24]:
with open('evasins_8cys.fasta', 'w') as outfile:
    for seq in cys_seqs:
        # The Sequence object from scikit-bio implements a 'write' method
        # that writes the sequence in FASTA format to an open file handle
        seq.write(outfile)

# Let's reassign the variable 'seqs' to point to our set with 8+ cysteines
seqs = cys_seqs

## Plotting with Plotly

In [25]:
!pip install plotly==2.2.1

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
# By importing Plotly this way we can avoid 'logging in' to use it
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [27]:
# We need to initialise offline plotly for the notebook so our charts display in-line
init_notebook_mode(connected=True)

### A histogram of sequence lengths

Plotly histograms: https://plot.ly/python/histograms/

We'd like to be able to visualize the distribution of sequence lengths in our evasin sequence set.

## Challenge

Write a **function** to return a list of lengths, given a list of sequences.

eg, we want to be able to write:
```
lengths = get_lengths(sequences)
```

## Solution

In [28]:
def get_lengths(sequences):
    """
    Returns a list of sequence lengths, given a list of sequences.
    """
    return [len(seq) for seq in sequences]


# or without a list comprehension
def get_lengths(sequences):
    lengths = []
    for seq in sequences:
        lengths.append(len(seq))
    return lengths

In [29]:
lengths = get_lengths(seqs)
lengths[0:5]

[]

In [30]:
bins = dict(size=1)
# bins = dict(start=0.5, end=300.5, size=1)
data = [go.Histogram(x=lengths, xbins=bins)]

iplot(data, filename='Evasin sequence length distribution')

The canonical evasins are ~120 - 140 residues long. Our set contains some sequences past 200 residues in length. This might warrant further investigation (but not today !).

Let's remove all the sequences longer than 300 residues.

In [ ]:
long_seqs = [seq.metadata['description'] for seq in seqs if len(seq) > 300]
print(long_seqs)

In [ ]:
sane_seqs = [seq for seq in seqs if len(seq) <= 300]
len(sane_seqs)

In [ ]:
bins = dict(size=1)
data = [go.Histogram(x=get_lengths(sane_seqs), xbins=bins)]

iplot(data, filename='Evasin sequence length distribution')